In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

In [2]:
html_page = requests.get("https://www.beeradvocate.com/beer/top-styles/")
soup = BeautifulSoup(html_page.content, 'html.parser')
styles=soup.find_all('div', {'class':'stylebreak'})

In [3]:
style_list = []
for i in range(len(styles)):
    style_list.append(styles[i].find_all('a')[::3])

In [4]:
link_list = []
for i in range(len(styles)):
    for n in range(len(styles[i].find_all('li'))):
        tmp = []
        tmp.append(styles[i].find('b').text) #style
        tmp.append(styles[i].find_all('li')[n].find('a').text.split('- ',1)[-1]) #substyle
        tmp.append(style_list[i][n].get('href'))  #link to top 100 of drink
        link_list.append(tmp)

In [5]:
beer_df=pd.DataFrame(link_list)
beer_df.columns = ['style', 'substyle', 'style_url']
beer_df['beer_urls'] = ''
beer_df.head()

,style,substyle,style_url,beer_urls
0,Bocks,Doppelbock,/beer/top-styles/35/,
1,Bocks,Eisbock,/beer/top-styles/36/,
2,Bocks,Maibock,/beer/top-styles/33/,
3,Bocks,Traditional,/beer/top-styles/32/,
4,Bocks,Weizenbock,/beer/top-styles/92/,


In [6]:
#Function to find the links and names for a specific style of beer.
#Takes in BS finding 'td' tags, taking only every 5th, or more simply, soup.find_all('td')[6::5]
#returns list of beer names and urls
def top_list(top_beers): 
    link_list = []
    for i in range(len(top_beers)):
        tmp=[]
        tmp.append(top_beers[i].find('a').text) #name
        tmp.append(top_beers[i].find('a')['href']) #url
        link_list.append(tmp)
    return link_list

In [7]:
for i in beer_df.index:
    time.sleep(2)
    style_url = 'https://www.beeradvocate.com{}'.format(beer_df['style_url'][i])
    html_page = requests.get(style_url)
    soup=BeautifulSoup(html_page.content, 'html.parser')
    top_beers = soup.find_all('td')[6::5]
    beer_df['beer_urls'][i] = top_list(top_beers)

In [ ]:
beer_df['beer_urls'][1]

In [8]:
beer_full_df = []
for i in beer_df.index:
    for n in range(len(beer_df['beer_urls'][i])):
        tmp = []
        tmp.append(beer_df['style'][i]) #style name
        tmp.append(beer_df['substyle'][i]) #substyle
        tmp.append(beer_df['beer_urls'][i][n][0])#beer name
        tmp.append(beer_df['beer_urls'][i][n][1])#url
        beer_full_df.append(tmp)

In [9]:
beer_full_df=pd.DataFrame(beer_full_df)
beer_full_df.columns = ['style','substyle', 'name', 'url']
beer_full_df.head()

,style,substyle,name,url
0,Bocks,Doppelbock,Ayinger Celebrator,/beer/profile/39/131/
1,Bocks,Doppelbock,Andechser Doppelbock Dunkel,/beer/profile/911/2899/
2,Bocks,Doppelbock,Korbinian,/beer/profile/252/779/
3,Bocks,Doppelbock,Troegenator - Bourbon Barrel-Aged,/beer/profile/694/112218/
4,Bocks,Doppelbock,Afternoon Delight,/beer/profile/20672/98588/


In [ ]:
beer_full_df.tail()

In [ ]:
beer_url = 'https://www.beeradvocate.com/beer/profile/467/2734/'
html_page = requests.get(beer_url)
soup=BeautifulSoup(html_page.content, 'html.parser')

In [ ]:
abv = soup.find_all('span', {'class':'Tooltip'})[2].text

In [ ]:
abv

In [10]:
review_df = []
for element in beer_full_df.index:
    time.sleep(2)
    beer_url = 'https://www.beeradvocate.com{}'.format(beer_full_df['url'][element])
    html_page = requests.get(beer_url)
    soup=BeautifulSoup(html_page.content, 'html.parser')
    abv = soup.find_all('span', {'class':'Tooltip'})[2].text
    review_soup = soup.find_all('div', {'id' : 'rating_fullview_container'})
    for review in range(len(review_soup)):
        tmp = []
        try: #not all reviews are complete, this'll skip them, and also not break the loop.
            tmp.append(review_soup[review]['ba-user'])#userID
            tmp.append(review_soup[review].find('b').next_sibling.split('from ', 1)[-1])#user location
            tmp.append(beer_full_df['name'][element])#beer name
            tmp.append(beer_full_df['url'][element]) #beer url
            tmp.append(beer_full_df['style'][element])#beer style
            tmp.append(beer_full_df['substyle'][element])#beer substyle
            tmp.append(review_soup[review].find('span', {'class':'BAscore_norm'}).text)#overall rating
            tmp.append(abv) #abv
            tmp2 = review_soup[review].find_all('span', {'class':'muted'})[1].text.split(' | ') #list of ratings
            tmp.append(tmp2[0].split(': ', 1)[-1])#look
            tmp.append(tmp2[1].split(': ', 1)[-1])#smell
            tmp.append(tmp2[2].split(': ', 1)[-1])#taste
            tmp.append(tmp2[3].split(': ', 1)[-1])#feel
            tmp.append(tmp2[4].split(': ', 1)[-1])#overall
            review_df.append(tmp)
        except:
            pass

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


IndexError: list index out of range

In [11]:
review_df=pd.DataFrame(review_df)
review_df.columns = ['userid', 'user_location', 'beer_name', 'beer_url',
                     'beer_style', 'beer_substyle', 
                     'overall_rating', 'abv', 'look', 
                     'smell', 'taste', 'feel', 'overall']

In [12]:
review_df

,userid,user_location,beer_name,beer_url,beer_style,beer_substyle,overall_rating,abv,look,smell,taste,feel,overall
0,1299270,Latvia,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,3.89,6.7%,3.75,4,3.75,4,4
1,226133,Michigan,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,4.79,6.7%,4.75,4.5,5,4.75,4.75
2,839604,Massachusetts,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,4.89,6.7%,4.5,4.75,5,4.75,5
3,1312814,Virginia,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,4.7,6.7%,4.25,4.75,4.75,4.5,4.75
4,1313117,Wisconsin,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,4.5,6.7%,4.5,4.5,4.5,4.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140829,495653,New York,Windlass of Erasmus,/beer/profile/54783/386131/,Wild/Sour Beers,Gose,4.25,6.5%,4.25,4.25,4.25,4.25,4.25
140830,601042,Colorado,Or It Gets The Gose Again (Cucumber),/beer/profile/27724/144139/,Wild/Sour Beers,Gose,4.09,3.1%,3.75,4,4.25,4,4
140831,887567,Florida,Or It Gets The Gose Again (Cucumber),/beer/profile/27724/144139/,Wild/Sour Beers,Gose,4.78,3.1%,4.5,4.5,5,4.75,4.75
140832,958449,Massachusetts,How To Reimagine The World,/beer/profile/45666/331015/,Wild/Sour Beers,Gose,3.6,5.3%,4.25,4.25,4.25,4.25,1


In [13]:
review_df.to_csv('beer_reviews3')

In [98]:
reviewer_df = review_df.drop(columns = ['beer_name', 'beer_url',
                     'beer_style', 'beer_substyle', 
                     'overall_rating', 'abv', 'look', 
                     'smell', 'taste', 'feel', 'overall'])

In [99]:
reviewer_df = reviewer_df.drop_duplicates(subset=['userid'])
reviewer_df

,userid,user_location
0,1299270,Latvia
1,226133,Michigan
2,839604,Massachusetts
3,1312814,Virginia
4,1313117,Wisconsin
...,...,...
140759,1237898,Nevada
140779,790909,Connecticut
140781,737135,New York
140804,1173220,Louisiana


In [92]:
beers_df = review_df.drop(columns = ['userid', 'user_location', 'overall_rating', 'look', 
                                     'smell', 'taste', 'feel', 'overall'])

In [95]:
beers_df = beers_df.drop_duplicates(subset=['beer_url'])
beers_df

,beer_name,beer_url,beer_style,beer_substyle,abv
0,Ayinger Celebrator,/beer/profile/39/131/,Bocks,Doppelbock,6.7%
25,Andechser Doppelbock Dunkel,/beer/profile/911/2899/,Bocks,Doppelbock,7.1%
50,Korbinian,/beer/profile/252/779/,Bocks,Doppelbock,7.4%
75,Troegenator - Bourbon Barrel-Aged,/beer/profile/694/112218/,Bocks,Doppelbock,11.5%
99,Afternoon Delight,/beer/profile/20672/98588/,Bocks,Doppelbock,10.1%
...,...,...,...,...,...
140783,Super You,/beer/profile/32893/290684/,Wild/Sour Beers,Gose,5.3%
140797,Lime Cucumber Gose,/beer/profile/45129/299156/,Wild/Sour Beers,Gose,4%
140805,Windlass of Erasmus,/beer/profile/54783/386131/,Wild/Sour Beers,Gose,6.5%
140830,Or It Gets The Gose Again (Cucumber),/beer/profile/27724/144139/,Wild/Sour Beers,Gose,3.1%


In [100]:
reviewer_df.to_csv('reviewers')

In [96]:
beers_df.to_csv('beers')

-----------------------------I really am changing the scope again aren't I. Back to using all of it. 

In [ ]:
'''
html_page = requests.get("https://www.beeradvocate.com/beer/top-styles/")
soup = BeautifulSoup(html_page.content, 'html.parser')
styles=soup.find_all('div', {'class':'stylebreak'})
'''

In [ ]:
'''
ipa_list = (styles[5].find_all('a')[::3]) #ipa is the 5th 

link_list = []
for n in range(len(ipa_list)):
    tmp = []
    #tmp.append(styles[i].find('b').text) #style
    tmp.append(ipa_list[n].text.split('- ',1)[-1]) #substyle?
    tmp.append(ipa_list[n]['href']) #link
    link_list.append(tmp)
'''

In [ ]:
'''
ipa_df=pd.DataFrame(link_list)
ipa_df.columns = ['style', 'style_url']
ipa_df['beer_urls'] = ''
ipa_df.head()
'''

In [ ]:
'''
#Function to find the links and names for a specific style of beer.
#Takes in BS finding 'td' tags, taking only every 5th, or more simply, soup.find_all('td')[6::5]
#returns list of beer names and urls
def top_list(top_beers): 
    link_list = []
    for i in range(len(top_beers)):
        tmp=[]
        tmp.append(top_beers[i].find('a').text)
        tmp.append(top_beers[i].find('a')['href'])
        link_list.append(tmp)
    return link_list
'''

In [ ]:
'''
for i in ipa_df.index:
    time.sleep(2)
    style_url = 'https://www.beeradvocate.com{}'.format(ipa_df['style_url'][i])
    html_page = requests.get(style_url)
    soup=BeautifulSoup(html_page.content, 'html.parser')
    top_beers = soup.find_all('td')[6::5]
    ipa_df['beer_urls'][i] = top_list(top_beers)
'''

In [ ]:
#ipa_df['beer_urls'][0][1]

In [ ]:
#ipa_df

In [ ]:
'''
ipa_full_df = []
for i in ipa_df.index:
    for n in range(len(ipa_df['beer_urls'][i])):
        tmp = []
        tmp.append(ipa_df['style'][i]) #style name
        tmp.append(ipa_df['beer_urls'][i][n][0])#beer name
        tmp.append(ipa_df['beer_urls'][i][n][1])#url
        ipa_full_df.append(tmp)
'''

In [ ]:
'''
ipa_full_df=pd.DataFrame(ipa_full_df)
ipa_full_df.columns = ['style', 'name', 'url']
ipa_full_df.head()
'''

I have an object that contains all of the beers I'm looking at, plus a link to where that beer is found. The next step is getting the reviews + reviewers for each. 

In [ ]:
'''
html_page = requests.get('https://www.beeradvocate.com/beer/profile/13014/138735/')
soup=BeautifulSoup(html_page.content, 'html.parser')
'''

In [ ]:
#reviews = soup.find_all('div', {'id' : 'rating_fullview_container'}) #Reviews are found here, 
                                                            #looks like the most recent 25 are available
                                                            #more reviews are available if signed in

In [ ]:
#reviews[0]

Location might matter since not all beers will be available in all locations. 

In [ ]:
#reviews[0].find('b').next_sibling.split('from ', 1)[-1] #location

In [ ]:
#reviews[1]

In [ ]:
#reviews[0]['ba-user'] #userid

In [ ]:
#reviews[0].find('span', {'class':'BAscore_norm'}).text #overall rating

In [ ]:
#reviews[0].find_all('span', {'class':'muted'})[1].text #phyiscal ratings? idk how best to describe them

In [ ]:
#test = reviews[0].find_all('span', {'class':'muted'})[1].text.split(' | ')
#for string in test:
    #print(string.split(': ', 1)[-1])
    #look, smell, taste, feel, overall

In [ ]:
'''
review_df = []
for element in ipa_full_df.index:
    time.sleep(2)
    beer_url = 'https://www.beeradvocate.com{}'.format(ipa_full_df['url'][element])
    html_page = requests.get(beer_url)
    soup=BeautifulSoup(html_page.content, 'html.parser')
    review_soup = soup.find_all('div', {'id' : 'rating_fullview_container'})
    for review in range(len(review_soup)):
        tmp = []
        try: #not all reviews are complete, this'll skip them, and also not break the loop.
            tmp.append(review_soup[review]['ba-user'])#userID
            tmp.append(review_soup[review].find('b').next_sibling.split('from ', 1)[-1])#user location
            tmp.append(ipa_full_df['name'][element])#beer name
            tmp.append(ipa_full_df['style'][element])#beer style
            tmp.append(reviews[0].find('span', {'class':'BAscore_norm'}).text)#overall rating
            tmp2 = review_soup[review].find_all('span', {'class':'muted'})[1].text.split(' | ') #list of ratings
            tmp.append(tmp2[0].split(': ', 1)[-1])#look
            tmp.append(tmp2[1].split(': ', 1)[-1])#smell
            tmp.append(tmp2[2].split(': ', 1)[-1])#taste
            tmp.append(tmp2[3].split(': ', 1)[-1])#feel
            tmp.append(tmp2[4].split(': ', 1)[-1])#overall
            review_df.append(tmp)
        except:
            pass
'''

In [ ]:
#review_df=pd.DataFrame(review_df)
#review_df.columns = ['userid', 'user_location', 'beer_name', 'beer_style', 'overall_rating', 'look', 'smell',
#                    'taste', 'feel', 'overall']

In [ ]:
#review_df.head()

In [ ]:
#review_df.to_csv('ipa_reviews')

CSV containing reviews, beer, and who did the reviewing. 

----------Start of old code and project start, keeping for reference.

In [ ]:
'''
html_page = requests.get("https://www.beeradvocate.com/beer/top-styles/")
soup = BeautifulSoup(html_page.content, 'html.parser')
'''

In [ ]:
'''
styles=soup.find_all('div', {'class':'stylebreak'})
len(styles)
#14 different styles.
'''

In [ ]:
'''
styles[0].find_all('a') #list of all the styles
#every 3rd entry is link to top 100 beers for each specific beer.
'''

In [ ]:
'''
for link in styles[0].find_all('a')[::3]:
    print(link.get('href'))
'''

In [ ]:
'''
style_list = []
for i in range(len(styles)):
    style_list.append(styles[i].find_all('a')[::3])
'''

In [ ]:
'''
counter=0
for i in range(len(style_list)):
    counter += len(style_list[i])
counter #113 different types of beers to review
'''

Looks like 113 different types of beer, that is a lot and likely outside the scope of this project, however I am going to continue. I can drop as the project continues. 

Creating a list of lists with all the links to each specific genre of beer, while maintaining the structure of the larger styles, to then convert into a PD dataframe with the style and genre. 

In [ ]:
'''
link_list = []
for i in range(len(style_list)):
    tmp = []
    for link in style_list[i]:
        tmp.append(link.get('href'))
    link_list.append(tmp)
'''

In [ ]:
'''
link_list = []
for i in range(len(styles)):
    for n in range(len(styles[i].find_all('li'))):
        tmp = []
        tmp.append(styles[i].find('b').text)
        tmp.append(styles[i].find_all('li')[n].find('a').text.split('- ',1)[-1])
        tmp.append(style_list[i][n].get('href'))
        link_list.append(tmp)
'''

Dataframe containing the style, substyle, and link to each.

In [ ]:
'''
beers_df=pd.DataFrame(link_list)
beers_df.columns = ['style', 'substyle', 'style_url']
beers_df['beer_urls'] = ''
beers_df.head()
'''

----------Testing Section to scrape

In [ ]:
'''
html_page = requests.get('https://www.beeradvocate.com/beer/top-styles/75/')
soup=BeautifulSoup(html_page.content, 'html.parser')
'''

In [ ]:
'''
top_beers = soup.find_all('td')[6::5]
#list of beers start at 6, and are every fifth value. 
#the top rated uses 100 beers, so with 113 different styles, and 100 each that's 11,000 different beers.
#Considering dropping whole styles of beer to make project more manageable.

top_beers[1].find('a')['href'] #link
top_beers[1].find('a').text #name
'''

----------------Ending testing section

In [ ]:
'''
def top_list(top_beers): 
    link_list = []
    for i in range(len(top_beers)):
        tmp=[]
        tmp.append(top_beers[i].find('a').text)
        tmp.append(top_beers[i].find('a')['href'])
        link_list.append(tmp)
    return link_list
#This creates a list of urls for the top 100 beers in a particular style. 
#Since I'll be using this a couple times, I'll make it a function
    #link_list
'''

In [ ]:
'''
for i in beers_df.index:
    time.sleep(2)
    style_url = 'https://www.beeradvocate.com{}'.format(beers_df['style_url'][i])
    html_page = requests.get(style_url)
    soup=BeautifulSoup(html_page.content, 'html.parser')
    top_beers = soup.find_all('td')[6::5]
    beers_df['beer_urls'][i] = top_list(top_beers)
'''

In [ ]:
styles[0].find_all('li')[1].find('a').text

----------------------The project scale is changing after this point. ---------------------------

This point I'm making the decision to cut the number of beers I'm using. If I continued on with this project at this size, I'll be looking at 10 or so different reviews for 12k different beers, so I'm going to continue the project using only IPAs. there's 7 different types of IPAs, which limits the project to about 700 different beers.  